In [3]:
# IMPORTAÇÕES
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json
from datetime import datetime

# Variáveis de Controle

min_para_parear = 10 # -> número de pessoas pareadas de modo fixo, sem usar os erros
num_pareamentos = 3 # -> número de pessoas pareadas ao final -> vai usar erros quadráticos para chegar nesse número

table_name = 'par_doutorandos_2014'

parear_por_area_conhecimento = False
parear_por_vinculo = False
parear_por_ano_doutorado_20 = False
parear_por_ano_doutorado_10 = False
parear_por_sexo = True
parear_por_regiao =  False
parear_por_erros = True
parear_por_grupo = [
    #"Graduações Iniciadas",
    "Orientações de Doutorado",
    "Premiações",
    #"Doutorados Iniciados",
    #"Pariticipação em Bancas de Graduação",
    #"Pós-Doutorados",
    #"Pariticipação em Bancas de Pós-Graduação",
    #"Cursos",
    "Atividades de Pesquisa e Desenvolvimento",
    "Produções Técnicas",
    "Serviços Técnicos/Especializados",
    "Atividades de Conselho, Direção ou Administração",
    "Orientações de Graduação",
    #"Eventos",
    "Pariticipação em Bancas - Outras",
    "Livros Publicados",
    "Atividades de Ensino",
    "Produções Bibliográficas",
    "Publicações de Artigos",
    "Vínculos Empregatícios ou Funcionais",
    "Apresentações de Trabalhos",
    "Mestrados",
    #"Mídias",
    "Patentes",
    "Participação em Congressos",
    #"Orientações de Pós-Doutorado",
    #"Livre Docência Obtida",
    "Orientações de Mestrado",
    #"Orientações - Outras",
    #"Outras Pós-Graduações lato sensu",
    "Anos de Doutor",
    'areas_conhecimento',
    ]

drop_table_if_exists = True


#Construindo SQL para criar tabela
sql = f'''
-- Table: public.{table_name}
'''
if drop_table_if_exists:
    sql += '''
DROP TABLE IF EXISTS public.{table_name};
'''

sql += f'''

CREATE TABLE IF NOT EXISTS public.{table_name}
(
    id bigint NOT NULL,
    ano integer,
    faixa "char",
    tipos_pareamento json,
    pareado_1 bigint,
    erro_1 numeric,
    pareado_2 bigint,
    erro_2 numeric,
    pareado_3 bigint,
    erro_3 numeric,
    pareamento_describe json,
    CONSTRAINT {table_name}_pkey1 PRIMARY KEY (id)
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public.{table_name}
    OWNER to postgres;
    
COMMENT ON TABLE public.pareamentos IS '
min_para_parear = {min_para_parear}
num_pareamentos = {num_pareamentos}

parear_por_area_conhecimento = {parear_por_area_conhecimento}
parear_por_vinculo = {parear_por_vinculo}
parear_por_ano_doutorado_20 = {parear_por_ano_doutorado_20}
parear_por_ano_doutorado_10 = {parear_por_ano_doutorado_10}
parear_por_sexo = {parear_por_sexo}
parear_por_regiao = {parear_por_regiao}

parear_por_erros = {", ".join(parear_por_grupo)}

table_name = {table_name}
drop_table_if_exists = {drop_table_if_exists}
';
'''

#Criando table, se não existir
db = Database.Database('CNPq')
engine = Carga.db_engine()
db.execute(sql)

#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())
dt = pd.read_sql("select * from indicadores_pareamento", engine)
print(f'dt.size: {dt.size}')
#size: 386.669.844      


#Pegando lista de pareamentos já realizados
print('Carregando lista de ids já feitos. Início em: ', datetime.now())
ids_já_feitos = []
sql = f'SELECT DISTINCT id FROM {table_name};'
ids_já_feitos = db.query(sql)
ids_já_feitos = [num[0] for num in ids_já_feitos] # Removendo lista dentro de lista por recuperar SQL
print(f'len(ids_já_feitos): {len(ids_já_feitos)}')

## Removendo os ids já feitos
dt = dt[~dt.id.isin(ids_já_feitos)]

print('Realizando normalização da tabela. Início em: ', datetime.now())


## Retirada dos não doutores - Existem 3 não doutores contemplados!!!
dt = dt[pd.notnull(dt.ano_doutorado)]


## Normalizando as Tabelas

#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

#Preenchendo Nan com "nenhum" nas áreas e vínculos
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

#Removendo acentos e caracteres especiais. Colocando tudo em minúsculas.
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()              

## Criando tabelas parciais
financiados = dt[~pd.isnull(dt.pgtos)]
dt = dt[pd.isnull(dt.pgtos)]
#erros = dt[dt.id.isin(financiados.id.unique())] #-> Listagem dos erros encontrados

## Alguns pesquisadores foram financiados em um ano, mas não em outro. Assim, aparece duas vezes, uma com pgtos=Nan, outra com pgtos > 0.
## Assim, é necessário retirar da lista de pareamento aqueles que foram financiados em algum momento.
## Da mesma forma, com certeza houveram aqueles financiados mais de um ano.
dt = dt[~dt.id.isin(financiados.id.unique())] #Percebi alguns erros, então tive a certeza de remover os financiados da tabela de pareamento


#
#
#
## Fazendo Pareamento
#
#
#



lista_pareamentos = [] # -> Lista com todos os pareados -> Útil apenas para conferência depois
lista_pareamentos_describe = []  #  -> lista com os describe de todos os pareamentos

número_ids_total = financiados.id.unique().size
número_ids_já_feitos = 0
tempo_início = datetime.now()
print(f'Iniciado em {tempo_início}.')

Carregando indicadores na memória. Início em:  2022-04-22 08:16:21.415189
dt.size: 111095490
Carregando lista de ids já feitos. Início em:  2022-04-22 08:18:33.092102
len(ids_já_feitos): 0
Realizando normalização da tabela. Início em:  2022-04-22 08:18:34.989760
Iniciado em 2022-04-22 08:19:21.014279.


In [12]:
financiados

,id,grupo,pgtos,sexo,uf,qty_2012,qty_2013,qty_2014,qty_2015,qty_2016,...,qty_2019,chamada,programa,area_demanda_bruta,areas_conhecimento,instituicoes,num_anos_na_instituicao,enquadramento_vinculo,tipos_vinculo,ano_doutorado
55,325690951570,Apresentações de Trabalhos,24287.32,F,RJ,119.0,125.0,137.0,141.0,147.0,...,166.0,Universal 142012 - Faixa B,Programa Basico de Geografia Física,geografia fisica,"engenharia civil, geociencias, geografia, geoe...","Universidade Federal do Rio de Janeiro, Ufrj I...","44, 15, 23, 32, 18, 14, 14, 13, 15, 30, 10, 10...","professor_titular, livre, livre, livre, livre,...","servidor_publico, colaborador, livre, livre, l...",1985
56,325690951570,"Atividades de Conselho, Direção ou Administração",24287.32,F,RJ,22.0,22.0,24.0,27.0,27.0,...,27.0,Universal 142012 - Faixa B,Programa Basico de Geografia Física,geografia fisica,"engenharia civil, geociencias, geografia, geoe...","Universidade Federal do Rio de Janeiro, Ufrj I...","44, 15, 23, 32, 18, 14, 14, 13, 15, 30, 10, 10...","professor_titular, livre, livre, livre, livre,...","servidor_publico, colaborador, livre, livre, l...",1985
57,325690951570,Atividades de Ensino,24287.32,F,RJ,20.0,20.0,20.0,20.0,20.0,...,24.0,Universal 142012 - Faixa B,Programa Basico de Geografia Física,geografia fisica,"engenharia civil, geociencias, geografia, geoe...","Universidade Federal do Rio de Janeiro, Ufrj I...","44, 15, 23, 32, 18, 14, 14, 13, 15, 30, 10, 10...","professor_titular, livre, livre, livre, livre,...","servidor_publico, colaborador, livre, livre, l...",1985
58,325690951570,Atividades de Pesquisa e Desenvolvimento,24287.32,F,RJ,17.0,20.0,18.0,20.0,20.0,...,20.0,Universal 142012 - Faixa B,Programa Basico de Geografia Física,geografia fisica,"engenharia civil, geociencias, geografia, geoe...","Universidade Federal do Rio de Janeiro, Ufrj I...","44, 15, 23, 32, 18, 14, 14, 13, 15, 30, 10, 10...","professor_titular, livre, livre, livre, livre,...","servidor_publico, colaborador, livre, livre, l...",1985
59,325690951570,Doutorados Iniciados,24287.32,F,RJ,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 142012 - Faixa B,Programa Basico de Geografia Física,geografia fisica,"engenharia civil, geociencias, geografia, geoe...","Universidade Federal do Rio de Janeiro, Ufrj I...","44, 15, 23, 32, 18, 14, 14, 13, 15, 30, 10, 10...","professor_titular, livre, livre, livre, livre,...","servidor_publico, colaborador, livre, livre, l...",1985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5049217,9998979664370790,Pós-Doutorados,45560.00,F,,2.0,2.0,2.0,2.0,2.0,...,2.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049218,9998979664370790,Premiações,45560.00,F,,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049219,9998979664370790,Produções Técnicas,45560.00,F,,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049220,9998979664370790,Publicações de Artigos,45560.00,F,,14.0,17.0,25.0,25.0,28.0,...,38.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9

In [13]:
id = 9998979664370790

In [14]:
    if not número_ids_já_feitos == 0:
        porcentagem_já_feita = (número_ids_já_feitos/número_ids_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / número_ids_já_feitos
        tempo_restante = (número_ids_total - número_ids_já_feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{número_ids_já_feitos}/{número_ids_total}. {porcentagem_já_feita * 100}% feitos. Fazendo id: {id}. Acabará em {tempo_em_que_vai_acabar}')
    else:
        print(f'{número_ids_já_feitos}/{número_ids_total}. Fazendo id: {id}.')
    número_ids_já_feitos += 1

    #Pegando dados do ID
    financiado = financiados[financiados.id == id]

2/12573. 0.01590710252127575% feitos. Fazendo id: 9998979664370790. Acabará em 2022-05-07 13:29:27.490979


In [15]:
financiado

,id,grupo,pgtos,sexo,uf,qty_2012,qty_2013,qty_2014,qty_2015,qty_2016,...,qty_2019,chamada,programa,area_demanda_bruta,areas_conhecimento,instituicoes,num_anos_na_instituicao,enquadramento_vinculo,tipos_vinculo,ano_doutorado
5049198,9998979664370790,Apresentações de Trabalhos,45560.0,F,,15.0,15.0,15.0,15.0,15.0,...,15.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049199,9998979664370790,"Atividades de Conselho, Direção ou Administração",45560.0,F,,5.0,6.0,6.0,7.0,7.0,...,10.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049200,9998979664370790,Atividades de Pesquisa e Desenvolvimento,45560.0,F,,12.0,11.0,11.0,14.0,14.0,...,12.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049201,9998979664370790,Cursos,45560.0,F,,10.0,10.0,10.0,12.0,12.0,...,13.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049202,9998979664370790,Doutorados Iniciados,45560.0,F,,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049203,9998979664370790,Eventos,45560.0,F,,66.0,71.0,75.0,84.0,90.0,...,116.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049204,9998979664370790,Graduações Iniciadas,45560.0,F,,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049205,9998979664370790,Livros Publicados,45560.0,F,,0.0,1.0,1.0,2.0,2.0,...,3.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049206,9998979664370790,Mestrados,45560.0,F,,1.0,1.0,1.0,1.0,1.0,...,1.0,Universal 2014 - Faixa B,PROGRAMA BASICO DE ZOOTECNIA,"avaliacao, producao e conservacao de forragens",nenhum,"Universidade Federal de Lavras, Revista Brasil...","25, 11, 10, 8, 9, 7, 6, 6, 4, 4, 3, 2, 3, 0, 2, 1","livre, livre, livre, livre, livre, livre, livr...","servidor_publico, livre, livre, livre, livre, ...",2007
5049207,9998979664370790,Orientações - Outras,45560.0,F,,0.0,0.

In [16]:
    
    #Pegando áreas do Conhecimento do Financiado
    financiado_areas = financiado.areas_conhecimento.iloc[0].split(',')

In [17]:
    
    #Pegando Regiões do Financiado
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado do Financiado -> Útil para calcular erro -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado do Financiado -> Inútil aqui -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa C', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando o vínculo do Financiado
    tipos_vinculo = financiado.tipos_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            é_servidor = 'servidor_publico'
        else:
            é_servidor = 'outros'   
    else: é_servidor = 'nenhum'
    
    #   
    #Fazendo o Pareamento
    #
    tipos_de_pareamento = []  #Variável que vai indicar quais os tipos de pareamento realizados
    pareados = dt
    
    #1. Pareando pela área de conhecimento
    if parear_por_area_conhecimento:
        pareados2 = pareados.loc[(pareados.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]
        if pareados2.id.unique().size > min_para_parear:
            tipos_de_pareamento.append('area_demanda_bruta')
            pareados = pareados2
                       
    #2. Pareando pelo Vínculo
    if parear_por_vinculo:
        if é_servidor == 'servidor_publico':
            pareados2 = pareados.loc[(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
        else:
            pareados2 = pareados.loc[~(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('tipos_vinculo')
                           
    
    #3. Pareando por ano de doutorado -> Faixa de 20 anos
    if parear_por_ano_doutorado_20:
        pareados2 = pareados.loc[
            (pareados.ano_doutorado < financiado.ano_doutorado.unique()[0] + 10) &
            (pareados.ano_doutorado > financiado.ano_doutorado.unique()[0] - 10)
            ]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('ano_doutorado_10')
                       
    #4. Pareando por ano de doutorado -> Faixa de 10 anos
    if parear_por_ano_doutorado_10:
        pareados2 = pareados.loc[
            (pareados.ano_doutorado < financiado.ano_doutorado.unique()[0] + 5) &
            (pareados.ano_doutorado > financiado.ano_doutorado.unique()[0] - 5)
            ]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('ano_doutorado_05')
                       
    #5. Pareando por sexo
    if parear_por_sexo:
        pareados2 = pareados.loc[(pareados.sexo == financiado.sexo.unique()[0])]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('sexo')
                       
    #6. Pareando por região
    if parear_por_regiao:
        if tipo_região == 1:
            pareados2 = pareados.loc[pareados.uf.isin(('SP',))]
        elif tipo_região == 2:
            pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
        elif tipo_região == 3:
            pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('uf')
                       
    #7. Pareando por erros quadráticos
    tipos_de_pareamento.append('erro')
              

In [40]:
ind_financiado.empty

True

In [1]:
dt

NameError: name 'dt' is not defined

In [46]:
    #Fezendo lista de erros quadráticos
    ids_pareados = []
    for index, row in pareados.iterrows():
        #Pegando o valor do Financiado
        ind_financiado = financiado.loc[(financiado.grupo == row.grupo)]
        if not ind_financiado.empty:
            ind_financiado = ind_financiado['qty_' + str(ano)].iloc[0]
            ind_pareado = row['qty_' + str(ano)]


            erro = {
                'id': id,
                'id_pareado': row.id,
                'grupo': row.grupo,
                'valor': ind_pareado,
                'erro2': (ind_financiado - ind_pareado)**2
            }

            ids_pareados.append(erro)

In [47]:
ids_pareados

[{'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Apresentações de Trabalhos',
  'valor': 6.0,
  'erro2': 81.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Atividades de Conselho, Direção ou Administração',
  'valor': 8.0,
  'erro2': 4.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Atividades de Pesquisa e Desenvolvimento',
  'valor': 0.0,
  'erro2': 121.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Cursos',
  'valor': 5.0,
  'erro2': 25.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Doutorados Iniciados',
  'valor': 1.0,
  'erro2': 0.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Eventos',
  'valor': 9.0,
  'erro2': 4356.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Graduações Iniciadas',
  'valor': 1.0,
  'erro2': 0.0},
 {'id': 9998979664370790,
  'id_pareado': 32202020745216,
  'grupo': 'Mestrados',
  

In [64]:
        #Acrescentando Anos de Doutor como um Indicador
        if "Anos de Doutor" in parear_por_grupo:
            for id_pareado in pareados.id.unique():
                ind_pareado = pareados.ano_doutorado.loc[
                    (pareados.id == id_pareado)
                    ].unique()
                #print(ind_pareado, type(ind_pareado))
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado[0]
                else:
                    ind_pareado = 0
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': "Anos de Doutor",
                    'valor': ind_pareado,
                    'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
                }
                ids_pareados.append(erro)

        #Acrescentando Áreas do Conhecimento em Comum como outro indicador
        if 'areas_conhecimento' in parear_por_grupo:
            areas_conhecimento_financiado = financiado.areas_conhecimento.iloc[0]
            if not areas_conhecimento_financiado == None:
                areas_conhecimento_financiado = areas_conhecimento_financiado.split(',')
                total_areas = len(areas_conhecimento_financiado)
    
                for id_pareado in pareados.id.unique():
                    areas_em_comum = 0
                    lista_areas_conhecimento = pareados.loc[pareados.id == id_pareado].areas_conhecimento
                    if len(lista_areas_conhecimento) > 0 and not lista_areas_conhecimento.iloc[0] == None:
                        lista_areas_conhecimento = lista_areas_conhecimento.iloc[0].split(',')
                        for area in areas_conhecimento_financiado:
                            if area in lista_areas_conhecimento:
                                areas_em_comum += 1
                    erro = {
                        'id': id,
                        'id_pareado': id_pareado,
                        'grupo': 'areas_conhecimento',
                        'valor': areas_em_comum,
                        'erro2': (total_areas - areas_em_comum)**2
                    }
                    ids_pareados.append(erro)  

In [65]:
    #Calculando o Desvio Padrão por id pareado
    erro = pd.DataFrame(ids_pareados)
    desvio_padrão = [] 
    if not erro.empty: 
        for id_pareado in erro.id_pareado.unique():
            erro_id = erro.loc[erro.id_pareado == id_pareado]
            if not erro_id.empty:
                soma_erro = erro_id.erro2.sum()
                num_erros = len(erro_id.index)
                err = {
                    'id_pareado': id_pareado,
                    'erro': np.sqrt(soma_erro/num_erros),
                }

                desvio_padrão.append(err)
                lista_pareamentos.append(err)

                #db.insert_dict(table_name, err, on_conflict=['id', 'id_pareado'])
        pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
        if not pareados_por_desvio_padrão.empty:
            lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:num_pareamentos]
            #lista_pareamentos_describe.append(pareados_por_desvio_padrão.erro.describe().to_json())
            #print(pareados_por_desvio_padrão.describe())
            #print(lista_pareados)
                      
    lista_pareado_para_incluir_no_bd = {
        'id': id,
        'ano': ano,
        'faixa': faixa,
        'tipos_pareamento': json.dumps(tipos_de_pareamento),
        'pareamento_describe': json.dumps(pareados_por_desvio_padrão.erro.describe().to_json())
    }
    x = 1
    for index, row in lista_pareados.iterrows():
        #print(row)
        lista_pareado_para_incluir_no_bd['pareado_' + str(x)] = row['id_pareado']
        lista_pareado_para_incluir_no_bd['erro_' + str(x)] = row['erro']
        x += 1
        if x > num_pareamentos: break
    db.insert_dict(table_name, lista_pareado_para_incluir_no_bd, on_conflict=['id'])

In [66]:

lista_pareado_para_incluir_no_bd


{'id': 9998979664370790,
 'ano': 2014,
 'faixa': 'B',
 'tipos_pareamento': '["area_demanda_bruta", "erro"]',
 'pareamento_describe': '"{\\"count\\":273.0,\\"mean\\":19.2610475441,\\"std\\":10.2693960089,\\"min\\":6.6749947982,\\"25%\\":13.9517024051,\\"50%\\":16.8726850264,\\"75%\\":20.2539433746,\\"max\\":71.3916852811}"',
 'pareado_1': 723462342997914.0,
 'erro_1': 6.674994798166929,
 'pareado_2': 986271844967281.0,
 'erro_2': 6.8594920787874205,
 'pareado_3': 1055722493921897.0,
 'erro_3': 7.532386215885585}

In [ ]:
                    ids_pareados.append(erro)

for id in financiados.id.unique():
    if not número_ids_já_feitos == 0:
        porcentagem_já_feita = (número_ids_já_feitos/número_ids_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / número_ids_já_feitos
        tempo_restante = (número_ids_total - número_ids_já_feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{número_ids_já_feitos}/{número_ids_total}. {porcentagem_já_feita * 100}% feitos. Fazendo id: {id}. Acabará em {tempo_em_que_vai_acabar}')
    else:
        print(f'{número_ids_já_feitos}/{número_ids_total}. Fazendo id: {id}.)
    número_ids_já_feitos += 1

    #Pegando dados do ID
    financiado = financiados[financiados.id == id]
    
    #Pegando áreas do Conhecimento do Financiado
    financiado.areas_conhecimento.iloc[0].split(',')
    
    #Pegando Regiões do Financiado
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado do Financiado -> Útil para calcular erro -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado do Financiado -> Inútil aqui -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa C', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando o vínculo do Financiado
    tipos_vinculo = financiado.tipos_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            é_servidor = 'servidor_publico'
        else:
            é_servidor = 'outros'   
    else: é_servidor = 'nenhum'
    
    #   
    #Fazendo o Pareamento
    #
    tipos_de_pareamento = []  #Variável que vai indicar quais os tipos de pareamento realizados
    pareados = dt
    
    #1. Pareando pela área de conhecimento
    if parear_por_area_conhecimento:
        pareados2 = pareados.loc[(pareados.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]
        if pareados2.id.unique().size > min_para_parear:
            tipos_de_pareamento.append('area_demanda_bruta')
            pareados = pareados2
                       
    #2. Pareando pelo Vínculo
    if parear_por_vinculo:
        if é_servidor == 'servidor_publico':
            pareados2 = pareados.loc[(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
        else:
            pareados2 = pareados.loc[~(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('tipos_vinculo')
                           
    
    #3. Pareando por ano de doutorado -> Faixa de 20 anos
    if parear_por_ano_doutorado_20:
        pareados2 = pareados.loc[
            (pareados.ano_doutorado < financiado.ano_doutorado.unique()[0] + 10) &
            (pareados.ano_doutorado > financiado.ano_doutorado.unique()[0] - 10)
            ]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('ano_doutorado_10')
                       
    #4. Pareando por ano de doutorado -> Faixa de 10 anos
    if parear_por_ano_doutorado_10:
        pareados2 = pareados.loc[
            (pareados.ano_doutorado < financiado.ano_doutorado.unique()[0] + 5) &
            (pareados.ano_doutorado > financiado.ano_doutorado.unique()[0] - 5)
            ]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('ano_doutorado_05')
                       
    #5. Pareando por sexo
    if parear_por_sexo:
        pareados2 = pareados.loc[(pareados.sexo == financiado.sexo.unique()[0])]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('sexo')
                       
    #6. Pareando por região
    if parear_por_regiao:
        if tipo_região == 1:
            pareados2 = pareados.loc[pareados.uf.isin(('SP',))]
        elif tipo_região == 2:
            pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
        elif tipo_região == 3:
            pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]
        if pareados2.id.unique().size > min_para_parear:
            pareados = pareados2
            tipos_de_pareamento.append('uf')
                       
    #7. Pareando por erros quadráticos
    tipos_de_pareamento.append('erro')
                                            
    #Fezendo lista de erros quadráticos
    ids_pareados = []
    for index, row in pareados.iterrows():
        id_pareado

        #Calculando os erros dos grupos
        # "grupo" é um dos grupos de indicadores.
        for grupo in financiado.grupo.unique():
            if grupo in parear_por_grupo:
                pareados_grupo = pareados.loc[(pareados.grupo == grupo)]

                #Pegando o valor do Financiado
                ind_financiado = financiado.loc[(financiado.grupo == grupo)]['qty_' + str(ano)].iloc[0]
    
                #Verificando o erro dos pareados restantes
                for id_pareado in pareados_grupo.id.unique():
                    ind_pareado = pareados_grupo.loc[(pareados.id == id_pareado)]['qty_' + str(ano)]
                    if len(ind_pareado) > 0:
                        ind_pareado = ind_pareado.iloc[0]
                    else:
                        ind_pareado = 0
                    erro = {
                        'id': id,
                        'id_pareado': id_pareado,
                        'grupo': grupo,
                        'valor': ind_pareado,
                        'erro2': (ind_financiado - ind_pareado)**2
                    }
                    ids_pareados.append(erro)

        #Acrescentando Anos de Doutor como um Indicador
        if "Anos de Doutor" in parear_por_grupo:
            for id_pareado in pareados.id.unique():
                ind_pareado = pareados.ano_doutorado.loc[
                    (pareados.id == id_pareado)
                    ].unique()
                #print(ind_pareado, type(ind_pareado))
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado[0]
                else:
                    ind_pareado = 0
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': "Anos de Doutor",
                    'valor': ind_pareado,
                    'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
                }
                ids_pareados.append(erro)

        #Acrescentando Áreas do Conhecimento em Comum como outro indicador
        if 'areas_conhecimento' in parear_por_grupo:
            areas_conhecimento_financiado = financiado.areas_conhecimento.iloc[0]
            if not areas_conhecimento_financiado == None:
                areas_conhecimento_financiado = areas_conhecimento_financiado.split(',')
                total_areas = len(areas_conhecimento_financiado)
    
                for id_pareado in pareados.id.unique():
                    areas_em_comum = 0
                    lista_areas_conhecimento = pareados.loc[pareados.id == id_pareado].areas_conhecimento
                    if len(lista_areas_conhecimento) > 0 and not lista_areas_conhecimento.iloc[0] == None:
                        lista_areas_conhecimento = lista_areas_conhecimento.iloc[0].split(',')
                        for area in areas_conhecimento_financiado:
                            if area in lista_areas_conhecimento:
                                areas_em_comum += 1
                    erro = {
                        'id': id,
                        'id_pareado': id_pareado,
                        'grupo': 'areas_conhecimento',
                        'valor': areas_em_comum,
                        'erro2': (total_areas - areas_em_comum)**2
                    }
                    ids_pareados.append(erro)  

    #Calculando o Desvio Padrão por id pareado
    erro = pd.DataFrame(ids_pareados)
    desvio_padrão = []
    if not erro.empty:
        desvio_padrão = []  # -> VARIÁVEL QUE VAI CONTER OS IDS PAREADOS
        for id_pareado in erro.id_pareado.unique():
            erro_id = erro.loc[erro.id_pareado == id_pareado]
            if not erro_id.empty:
                soma_erro = erro_id.erro2.sum()
                num_erros = len(erro_id.index)
                err = {
                    'id_pareado': id_pareado,
                    'erro': np.sqrt(soma_erro/num_erros),
                }
                
                desvio_padrão.append(err)
                lista_pareamentos.append(err)

                #db.insert_dict(table_name, err, on_conflict=['id', 'id_pareado'])
        pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
        if not pareados_por_desvio_padrão.empty:
            lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:num_pareamentos]
            #lista_pareamentos_describe.append(pareados_por_desvio_padrão.erro.describe().to_json())
            #print(pareados_por_desvio_padrão.describe())
            #print(lista_pareados)
                      
    lista_pareado_para_incluir_no_bd = {
        'id': id,
        'ano': ano,
        'faixa': faixa,
        'tipos_pareamento': json.dumps(tipos_de_pareamento),
        'pareamento_describe': json.dumps(pareados_por_desvio_padrão.erro.describe().to_json())
    }
    x = 1
    for index, row in lista_pareados.iterrows():
        #print(row)
        lista_pareado_para_incluir_no_bd['pareado_' + str(x)] = row['id_pareado']
        lista_pareado_para_incluir_no_bd['erro_' + str(x)] = row['erro']
        x += 1
        if x > num_pareamentos: break
    db.insert_dict(table_name, lista_pareado_para_incluir_no_bd, on_conflict=['id'])
    
porcentagem_já_feita = (número_ids_já_feitos/número_ids_total)
print(f'''

Tarefa Terminada.

{número_ids_já_feitos}. {porcentagem_já_feita * 100}% feitos. 
{ids_já_feitos} pulados no início. 
Último feito: {id}. 
Iniciado em: {tempo_início}.
Acabou em {datetime.now()}.
Tempo por id analisado: {(datetime.now() - tempo_início)/número_ids_já_feitos} segundos.

Criando tabela Excel com desvios padrões:
''')

In [ ]:
#Carregando pareamentos realizados
pareados = pd.read_sql(table_name, engine)

#Carregar arquivo na memória
dt = pd.read_sql(f"select * from public.indicadores_pareamento where not ano_doutorado is null and id in (SELECT distinct(unnest(array[id, pareado_1, pareado_2, pareado_3])) AS id FROM public.{table_name})", engine)

## Retirada dos não doutores - Existem 3 não doutores contemplados!!!
dt = dt[pd.notnull(dt.ano_doutorado)]


## Normalizando as Tabelas

#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

#Preenchendo Nan com "nenhum" nas áreas e vínculos
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

#Removendo acentos e caracteres especiais. Colocando tudo em minúsculas.
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()              

## Criando tabelas parciais
financiados = dt[~pd.isnull(dt.pgtos)]
dt = dt[pd.isnull(dt.pgtos)]


## Alguns pesquisadores foram financiados em um ano, mas não em outro. Assim, aparece duas vezes, uma com pgtos=Nan, outra com pgtos > 0.
## Assim, é necessário retirar da lista de pareamento aqueles que foram financiados em algum momento.
## Da mesma forma, com certeza houveram aqueles financiados mais de um ano.
dt = dt[~dt.id.isin(financiados.id.unique())] #Percebi alguns erros, então tive a certeza de remover os financiados da tabela de pareamento

## Criando Excel Writer
path = f"C:/Users/silva/CNPq/Lattes/Tabela_Melaine_{table_name}.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
    
# Pegando tabela com contagem de grupos
lista_indicadores = []
for grupo in dt.grupo.unique():
    dt_grupo = dt[dt.grupo == grupo]
    financiados_grupo = financiados[financiados.grupo == grupo]
    tabela = {
        'grupo': grupo,
        'financiados_média': financiados_grupo.qty_2012.mean(),
        'pareados_média': dt_grupo.qty_2012.mean(),
        'financiados_desvio_padrão': financiados_grupo.qty_2012.std(),
        'pareados_desvio_padrão': dt_grupo.qty_2012.std()
    }
    lista_indicadores.append (tabela)

## Tabela Ano Doutorado
financiados_dt = financiados[['id', 'ano_doutorado']].drop_duplicates() 
dt_dt = dt[['id', 'ano_doutorado']].drop_duplicates()  
tabela = {
    'grupo': 'ano_doutorado',
    'financiados_média': financiados_dt.ano_doutorado.mean(),
    'pareados_média': dt_dt.ano_doutorado.mean(),
    'financiados_desvio_padrão': financiados_dt.ano_doutorado.std(),
    'pareados_desvio_padrão': dt_dt.ano_doutorado.std()
}
lista_indicadores.append (tabela)


### Salvando a primeira planilha no Writer
tabela = pd.DataFrame(lista_indicadores)
tabela.to_excel(writer, sheet_name = 'Grupos')

## Tabela Sexo
financiados_sexo = financiados[['id', 'sexo']].drop_duplicates() 
dt_sexo = dt[['id', 'sexo']].drop_duplicates()  
fsm = financiados_sexo[financiados_sexo.sexo == 'M'].id.count()
fsf = financiados_sexo[financiados_sexo.sexo == 'F'].id.count()
dsm = dt_sexo[dt_sexo.sexo == 'M'].id.count()
dsf = dt_sexo[dt_sexo.sexo == 'F'].id.count()
tabela = [['Descrição', 'Valor']]
tabela.append(['grupo', 'sexo'])
tabela.append(['financiados: homens %', fsm/(fsm + fsf)])
tabela.append(['pareados: homens %', dsm/(dsm + dsf)])
tabela.append(['total_financiados', financiados_sexo.id.count()])
tabela.append(['total_pareados', dt_sexo.id.count()])
sexo = pd.DataFrame(tabela)
sexo.to_excel(writer, sheet_name = 'Sexo')

## Salvando o arquivo
writer.save()
writer.close()
print(f'Tabela criada. Fim dos serviços. Finalização em {datetime.now()}')


697╣